In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical


#MNIST 데이터셋을 로드하여 준비. 샘플 값을 정수에서 부동소수로 변환
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# 28x28차원의 벡터가 60000개 채널은 1개
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))
train_images, test_images = train_images / 255.0, test_images / 255.0

from tensorflow.keras.utils import to_categorical

one_hot_train_labels = to_categorical(train_labels, 10)
one_hot_test_labels = to_categorical(test_labels, 10)
print(one_hot_train_labels[:10])


# Functional API를 이용한 모델 구현
# tensorflow.keras.models.Model의 inputs() & outputs() 메소드 설정을 이용해 모델 정의x`
# (1) Convolution layer

inputs = tf.keras.Input(shape=(28*28, ))
x=Dense(input_dim=784,units=256,activation='relu')(inputs)
y=Dense(input_dim=784,units=256,activation='relu')(inputs)
C=x+y
outputs=Dense(input_dim=256,units=10,activation='sigmoid') (C)
model=Model(inputs=inputs,outputs=outputs)

# 모델 학습과정 설정
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 구조 요약 출력
model.summary()

# 모델 학습
history = model.fit(train_images, one_hot_train_labels, epochs=5, batch_size=10)

print("\n=============test results==========")
labels=model.predict(test_images)
print("\n Accuracy: %.4f" % (model.evaluate(test_images, one_hot_test_labels, verbose=2)[1
])) #list index

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.datasets import mnist

n_output = 784
n_noise = 128

generator = Sequential()
generator.add(Dense(units=256, input_dim=n_noise, activation='relu'))
generator.add(Dense(units=512, activation='relu'))
generator.add(Dense(units=n_output, activation='tanh'))

discriminator = Sequential()
discriminator.add(Dense(256, input_dim=n_output, activation='relu'))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(1, activation='sigmoid'))

g_input = Input(shape=(n_noise,))
g_output = discriminator(generator(g_input))
gan = Model(g_input, g_output)

discriminator.trainable = True
adam = tf.optimizers.legacy.Adam(lr=0.0002, beta_1=0.5)
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

discriminator.trainable = False
gan.compile(loss='binary_crossentropy', optimizer=adam)

gan.summary()

n_epoch = 100
batch_size = 128
saving_interval = 10

(train_images, _), (_, _) = mnist.load_data()

buffer_size = len(train_images)
train_images = train_images.reshape((buffer_size, n_output))

train_images = (train_images - 127.5) / 127.5

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(buffer_size).batch(batch_size)

for i in  range(n_epoch):
  print('Epoch Num : {}/{}'.format((i + 1), n_epoch))

  for image_batch in train_dataset:
    n_bt_imgs = len(image_batch)
    true_labels = np.ones((n_bt_imgs, 1))
    fake_labels = np.ones((n_bt_imgs, 1))

    d_loss_real=discriminator.train_on_batch(image_batch,true_labels)

    noise = np.random.normal(0, 1, (n_bt_imgs, n_noise))
    gen_imgs = generator.predict(noise)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    g_loss = gan.train_on_batch(noise, true_labels)

    if (i + 1) % saving_interval == 0 or i == 1:
      print('Epoch:%d' % (i + 1),
            'd_loss:%.4f' % d_loss, 'g_loss:%.4f' % g_loss)

      noise = np.random.normal(0, 1, (25, n_noise))
      gen_imgs = generator.predict(noise)

      gen_imgs = 0.5 * gen_imgs + 0.5

      fig, axs = plt.subplots(5, 5)
      count = 0
      for j in range(5):
        for k in range(5):
          axs[j, k].imshow(np.reshape(gen_imgs[count], (28, 28)))
          axs[j, k].axis('off')
          count += 1
      plt.show()


In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Dropout
from tensorflow.keras.models import Sequential, Model
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

n_output = 784   # 28×28 이미지 크기
n_noise = 128    # 랜덤 노이즈 벡터 크기

# 생성자 모델 정의
# 128차원의 랜덤 노이즈 벡터를 입력받아서 784차원의 벡터(28×28)이미지를 출력
generator = Sequential()
generator.add(Dense(units=256 , input_dim=n_noise))
generator.add(LeakyReLU(0.2))
generator.add(Dense(units=512 ))
generator.add(LeakyReLU(0.2))
generator.add(Dense(units=1024))
generator.add(LeakyReLU(0.2))
generator.add(Dense(units=n_output, activation='tanh'))

# 판별자 모델 정의
discriminator = Sequential()
discriminator.add(Dense(1024, input_dim=n_output))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(1, activation='sigmoid'))

# 생성자와 판별자 모델을 통합한 GAN 모델 구현
g_input = Input(shape=(n_noise,))
g_output = discriminator(generator(g_input))
gan = Model(g_input, g_output)

# 판별자 모델 학습을 위한 최적화 함수 및 손실 함수 설정
discriminator.trainable = True
adam = tf.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.5)
discriminator.compile(loss='binary_crossentropy', optimizer=adam)


# GAN 모델 학습을 위한 최적화 함수 및 손실 함수 설정
# GAN 모델에 대한 학습은 오직 생성자에 대한 가중치 업데이트가 수행되어야 하므로 판별자 모델의 학습 여부를 False로 설정함
discriminator.trainable = False
gan.compile(loss='binary_crossentropy', optimizer=adam)

# GAN 모델 구조 출력
gan.summary()

# 모델 학습
n_epoch = 100
batch_size = 128
saving_interval = 10

# MNIST 데이터 불러오기
# 모델 학습을 위해서는 훈련 이미지만 사용할 것이기 때문에 train_images만 로드
(train_images, _), (_, _) = mnist.load_data()

buffer_size = len(train_images)  # 60000
train_images = train_images.reshape((buffer_size, n_output)) #28x28

# 이미지 픽셀값을 [–1~1]로 정규화
train_images = (train_images - 127.5) / 127.5

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(buffer_size).batch(batch_size)

for i in range(n_epoch):
    print('Epoch Num : {}/{}'.format((i + 1), n_epoch))

    # 1) 미니-배치별 모델 가중치 갱신 및 오차 계산
    for image_batch in train_dataset:    # 미니-배치별 가중치 갱신
        # 현재 배치 크기만큼 진짜 및 가짜 레이블 배열 생성
        n_bt_imgs = len(image_batch)
        true_labels = np.ones((n_bt_imgs, 1))  # 진짜 레이블(1) 생성
        fake_labels = np.zeros((n_bt_imgs, 1)) # 가짜 레이블(0) 생성

        # 현재 배치에 대한 판별자 모델의 가중치 1회 갱신 및 오차 갱신
        # 진짜 이미지에 대한 판별자 모델 가중치 갱신 및 오차 계산
        d_loss_real = discriminator.train_on_batch(image_batch, true_labels)
        # 가짜 이미지에 대한 판별자 모델 가중치 갱신 및 오차 계산
        noise = np.random.normal(0, 1, (n_bt_imgs, n_noise))
        gen_imgs = generator.predict(noise)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # 현재 배치에 대한 GAN 모델 가중치 1회 갱신 및 오차 계산
        g_loss = gan.train_on_batch(noise, true_labels)

    # 2) 중간 과정 생성되는 이미지 확인
    if (i +1) % saving_interval == 0 or i == 1:
        print('Epoch:%d' % (i + 1),
             'd_loss:%.4f' % d_loss, ' g_loss:%.4f' % g_loss)

        # 학습된 생성자 모델을 이용하여 25개의 이미지 생성
        noise = np.random.normal(0, 1, (25, n_noise))  # 25개의 노이즈 벡터 생성
        gen_imgs = generator.predict(noise)  # 생성자를 이용한 25개의 이미지 생성

        # 이미지 픽셀값 0~1로 조정
        #gen_imgs = 0.5 * gen_imgs + 0.5

       # 생성된 이미지 출력 및 저장
        fig, axs = plt.subplots(5, 5)
        count = 0
        for j in range(5):
            for k in range(5):
                axs[j, k].imshow(np.reshape(gen_imgs[count], (28, 28)))
                axs[j, k].axis('off')
                count += 1
        plt.show()
        #fig.savefig("gan_results/g_mnist_%d.png" % (i + 1))

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
# Style Transfer


import tensorflow as tf
from PIL import Image #이미지 분석 및 처리 Pillow 라이브러리
import numpy as np
import matplotlib.pyplot as plt
#TensorFlow Hub는 재사용 가능한 머신러닝을 위한 개방형 리포지토리 및 라이브러리
import tensorflow_hub as hub
import functools
import os

#load image
def load_image(image_path):
  image = tf.io.read_file(image_path)
  image = tf.image.decode_image(image, channels=3)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, (256,256))
  #tr.newaxis를 이용한 reshape방법. 맨 앞에 shape 1이 추가 (원래 image(256,256,3))=>(1,256,256,3)
  image = image[tf.newaxis,:]
  #print(image)
  return image

def tensor2image(image):
  image = image*255.0
  image = np.array(image, dtype=np.uint8)
  return Image.fromarray(image)

content_image = load_image('/content/gdrive/My Drive/content1.png')
style_image = load_image('/content/gdrive/My Drive/style1.jpg')

hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
stylized_image = hub_module(tf.constant(content_image), tf.constant(style_image))[0][0]
#stylized_image = outputs[0][0]

final_image = tensor2image(stylized_image)
if final_image.mode != 'RGB':
    final_image = final_image.convert('RGB')
final_image.save('/content/gdrive/My Drive/stylized_image1.jpg')